<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFE_network_w_Dack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%time 
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

In [0]:
# Load the Drive helper and mount
from google.colab import drive
%tensorflow_version 2.x
# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
%%time
import datetime
import glob
import math
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import networkx as nx
from shapely.geometry import Point
from sys import getsizeof
import dask.dataframe as dd
from dask.distributed import Client
%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'

CPU times: user 271 ms, sys: 55.8 ms, total: 327 ms
Wall time: 947 ms


In [0]:
client = Czlient(processes=False)
client

In [0]:
path = '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_input'
path_output = '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_output'

!ls "/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_input"

buildings_raw.cpg  buildings_raw.prj  buildings_raw.shp  GD_wind.csv
buildings_raw.dbf  buildings_raw.qpj  buildings_raw.shx


In [0]:
def load_data(file_name, minx, miny, maxx, maxy):
    # crop data
    bbox = box(minx, miny, maxx, maxy)
    # building point dataset
    gdf_buildings = gpd.read_file(os.path.join(path, file_name), bbox=bbox)
    print(gdf_buildings.dtypes)
    max_extent = gdf_buildings.total_bounds
    data_size = getsizeof(gdf_buildings)/(1024.0**3)
    print("Shapefile extent : {}".format(max_extent))
    print("Asset loaded : {}".format(len(gdf_buildings)))
    print("Data size:{} GB'".format(data_size))
    # gdf_buildings.IgnProb_bl = 0.02
    # xmin,ymin,xmax,ymax = gdf_buildings.total_bounds
    # Precision of float32 is sufficient for lat and lon
    float_columns = ['SHAPE_Leng','SHAPE_Area',
                    'IgnProb_bl','RandProb']
    gdf_buildings[float_columns] = gdf_buildings[float_columns].astype('float32')
    int_columns = ['TARGET_FID','Combustibl',
                    'AU2013Num','RandProb']
    gdf_buildings[int_columns] = gdf_buildings[int_columns].astype('int32')
    data_size = getsizeof(gdf_buildings)/(1024.0**3)
    print("resized Data size:{} GB'".format(data_size))
    return gdf_buildings


def wind_scenario():
    wind_data = pd.read_csv(os.path.join(path, 'GD_wind.csv'))
    i = np.random.randint(0, wind_data.shape[0])
    w = wind_data.iloc[i, 2]
    d = wind_data.iloc[i, 1]
    b = wind_data.iloc[i, 3]
    return w, d, b


def create_network(edge_list_dataframe):
    graph = nx.from_pandas_edgelist(edge_list_dataframe, edge_attr=True)
    # options = {'node_color': 'red', 'node_size': 50, 'width': 1, 'alpha': 0.4,
    #            'with_labels': False, 'font_weight': 'bold'}
    # nx.draw_kamada_kawai(graph, **options)
    # plt.show()
    return graph

In [0]:
def build_edge_list_dack(geodataframe, maximum_distance, polygon_file):
    # create arrays for different id combination
    n = np.arange(0, len(geodataframe))
    target = [n] * len(geodataframe)
    target = np.hstack(target)
    source = np.repeat(n, len(geodataframe))
    # put arrays in dataframe
    df = pd.DataFrame()
    df['source_id'] = source
    df['target_id'] = target
    # merge source attributes with source index
    geo_df = geodataframe.copy()
    geo_df['id'] = geo_df.index
    # create source / target gdf from gdf.columns of interest
    geo_df = geo_df[['id', 'TARGET_FID', 'X', 'Y', 'geometry', 'IgnProb_bl']]
    geo_df_TRG = geo_df.copy()
    geo_df_TRG.columns = ['target_' + str(col) for col in geo_df_TRG.columns]
    geo_df_SRC = geo_df.copy()
    geo_df_SRC.columns = ['source_' + str(col) for col in geo_df_SRC.columns]
    # merge data
    merged_data = pd.merge(df, geo_df_SRC, left_on='source_id', right_on='source_id', how='outer')
    merged_data = pd.merge(merged_data, geo_df_TRG, left_on='target_id', right_on='target_id', how='outer')
    merged_data.rename(columns={'source_id': 'source', 'target_id': 'target'}, inplace=True)
    # calculate distance for each source / target pair
    # create a df from polygon shape to get accurate distance
    # print(list(polygon_file))
    polygon = polygon_file[['TARGET_FID', 'geometry']]
    # print(list(polygon))
    source_poly = merged_data[['source_TARGET_FID']]
    target_poly = merged_data[['target_TARGET_FID']]
    # print(list(source_poly))
    src_poly = pd.merge(source_poly, polygon, left_on='source_TARGET_FID', right_on='TARGET_FID', how='left')
    trg_poly = pd.merge(target_poly, polygon, left_on='target_TARGET_FID', right_on='TARGET_FID', how='left')
    src_poly_gdf = gpd.GeoDataFrame(src_poly, geometry='geometry')
    trg_poly_gdf = gpd.GeoDataFrame(trg_poly, geometry='geometry')
    distance_series = src_poly_gdf.distance(trg_poly_gdf)
    # print(distance_series)

    # insert distance in merged data column
    merged_data['v1'] = merged_data.source_X - merged_data.target_X
    merged_data['v2'] = merged_data.source_Y - merged_data.target_Y
    # merged_data['euc_distance'] = np.hypot(merged_data.v1, merged_data.v2)
    merged_data['euc_distance'] = distance_series
    # remove when distance "illegal"
    valid_distance = merged_data['euc_distance'] < maximum_distance
    not_same_node = merged_data['euc_distance'] != 0
    data = merged_data[valid_distance & not_same_node]
    # calculate azimuth
    data['azimuth'] = np.degrees(np.arctan2(merged_data['v2'], merged_data['v1']))
    data['bearing'] = (data.azimuth + 360) % 360
    return data

In [0]:
def set_initial_fire_to(df):
    """Fine = 0, Fire = 1, Burned = 2"""
    df['RNG'] = np.random.uniform(0, 1, size=len(df))  # add for random suppression per building, df.shape[0])
    onFire = df['source_IgnProb_bl'] > df['RNG']
    ignitions = df[onFire]
    # source nodes ignited
    sources_on_fire = list(ignitions.source)
    sources_on_fire = list(dict.fromkeys(sources_on_fire))
    return sources_on_fire


def set_fire_to(df, existing_fires):
    are_set_on_fire = (df['source'].isin(existing_fires))
    spark = df[are_set_on_fire]
    # source nodes ignited
    sources_on_fire = list(spark.source)
    sources_on_fire = list(dict.fromkeys(sources_on_fire))
    return sources_on_fire


def fire_spreading(list_fires, list_burn, wind_speed, wind_bearing, suppression_threshold, step_value, data):
    # check the fire potential targets
    # print("fire list before spreading : {}, length : {}".format(list_fires, len(list_fires)))
    are_potential_targets = (data['source'].isin(list_fires))
    are_not_already_burned = (~data['target'].isin(list_burn))
    df = data[are_potential_targets & are_not_already_burned]
    if df.empty:
        # print("no fires")
        list_burn.extend(list(list_fires))
        list_burn = list(dict.fromkeys(list_burn))
        return [], list_burn  # to break the step loop
    # set up additional CONDITIONS for fire spreading

    # neighbors selection from buffer
    are_neighbors = df['distance'] < wind_speed
    df = df[are_neighbors]

    # wind direction
    wind_bearing_max = wind_bearing + 45
    wind_bearing_min = wind_bearing - 45
    if wind_bearing == 360:
        wind_bearing_max = 45
    if wind_bearing <= 0:  # should not be necessary
        wind_bearing_min = 0
    if wind_bearing == 999:
        wind_bearing_max = 999
        wind_bearing_min = 0
    are_under_the_wind = (df['bearing'] < wind_bearing_max) & (df['bearing'] > wind_bearing_min)
    # print("targets under the wind ? {}".format(list(dict.fromkeys(list(are_under_the_wind)))))
    df = df[are_under_the_wind]
    # suppression
    df['random'] = np.random.uniform(0, 1, size=len(df))
    are_not_suppressed = df['random'] > suppression_threshold
    # print("fire suppressed ? {}".format(list(dict.fromkeys(list(are_not_suppressed)))))
    df = df[are_not_suppressed]

    # spread fire based on condition
    fire_df = df
    # fire_df = df[are_neighbors & are_under_the_wind & are_not_suppressed]  # issues with "are_under_the_wind
    # print(len(fire_df.head(5)))
    # print(len(fire_df))
    list_burn.extend(list(list_fires))
    fire_df['step'] = step_value
    # fire_df.to_csv(os.path.join(path_output, "step{}_fire.csv".format(step_value))) # ADD IF CSV OUTPUT NEEDED
    list_fires = list(dict.fromkeys(list(fire_df.target)))
    list_burn.extend(list(fire_df.target))
    list_burn = list(dict.fromkeys(list_burn))
    return list_fires, list_burn


def log_files_concatenate(prefix, scenario_count):
    list_df = []
    files = glob.glob(os.path.join(path_output, prefix))
    if files:
        for file in files:
            # print(file)
            df = pd.read_csv(os.path.join(path_output, file))
            list_df.append(df)
            os.remove(file)
        data = pd.concat(list_df)
        data['scenario'] = scenario_count
        data.to_csv(os.path.join(path_output, "fire_scenario_{}.csv".format(scenario_count)))
    else:
        print("no files to concatenate")


def clean_up_file(prefix, path_path=path_output):
    files = glob.glob(os.path.join(path_path, prefix))
    for file in files:
        # print(file)
        os.remove(file)

In [0]:
def main(number_of_scenarios, edges):
  clean_up_file("*csv")
  clean_up_file("*png")
  scenarios_list = []
  log_burned = []  # no removing duplicate
  # --- SCENARIOS
  t = datetime.datetime.now()
  print("number of scenarios : {}".format(number_of_scenarios))
  for scenario in range(number_of_scenarios):
      t0 = datetime.datetime.now()
      burn_list = []
      print("--- SCENARIO : {}".format(scenario))
      fire_list = set_initial_fire_to(edges)
      if len(fire_list) == 0:
          print("no fire")
          log_burned.extend(burn_list)
          scenarios_list.extend([scenario] * len(burn_list))
          continue
      w_direction, w_speed, w_bearing = wind_scenario()
      # --------- STEPS
      for step in range(len(edges)):
          print("--------- STEP : {}".format(step))
          fire_list = set_fire_to(edges, fire_list)
          fire_list, burn_list = fire_spreading(fire_list, burn_list, w_speed, w_bearing, 0, step, edges)
          if len(fire_list) == 0:
              break
      log_burned.extend(burn_list)
      scenarios_list.extend([scenario] * len(burn_list))

      # log_files_concatenate('step*', scenario)
      
  return scenarios_list, log_burned

In [0]:
def postprocessing(scenarios_recorded, burned_asset, gdf_polygons):
    list_of_tuples = list(zip(scenarios_recorded, burned_asset))
    df = pd.DataFrame(list_of_tuples, columns=['scenarios', 'burned_asset_index'])
    # df burn count per asset
    df['count'] = df.groupby('burned_asset_index')['burned_asset_index'].transform('count')
    print(df.describe())
    df = df[['burned_asset_index', 'count']]
    df_count = pd.merge(df, gdf_polygon, left_on='burned_asset_index', right_on='TARGET_FID', how='left')
    # to geodataframe
    crs = gdf_polygon.crs
    gdf_count = gpd.GeoDataFrame(df_count, crs=crs, geometry='geometry')
    # plot
    fig, ax = plt.subplots(1, 1)
    gdf_count.plot(column='count', cmap='RdYlBu_r', ax=ax, legend=True)
    ax.title.set_text("Burned buildings after {} scenarios".format(max(scenarios_recorded)+1))
    plt.tight_layout()
    plt.savefig(os.path.join(path_output, "results_{}.png".format(max(scenarios_recorded)+1)))
    plt.show()
    plt.close(fig)
    gdf_count = gdf_count.drop(columns=['SHAPE_Leng', 'SHAPE_Area', 'AU2013Num', 'IgnProb_bl', 'RandProb'])
    df_count.to_csv(os.path.join(path_output, "results_{}_scenarios.csv".format(max(scenarios_recorded)+1)))
    gdf_count.to_file(os.path.join(path_output, "results_{}_scenarios.shp".format(max(scenarios_recorded)+1)))

    
    return df_count